# DOCommunication - Análisis de Grafos de Comunicación Gestual

Este notebook demuestra el análisis completo de interacciones gestuales en UCI usando teoría de grafos.

**Autor**: DOCommunication Team  
**Versión**: 1.0  
**Fecha**: 2024  

## Contenido

1. Configuración e Imports
2. Generación de Datos Simulados
3. Construcción del Grafo
4. Análisis de Métricas Estructurales
5. Análisis de Centralidades
6. Detección de Comunidades
7. Análisis de Robustez
8. Análisis de Transiciones
9. Modelos de Difusión
10. Visualizaciones

## 1. Configuración e Imports

In [ ]:
# Imports
import sys
from pathlib import Path

# Agregar src al path
backend_path = Path.cwd().parent
src_path = backend_path / "src"
sys.path.insert(0, str(src_path))

# Imports del proyecto
from src.graph.analyzer import InteractionGraphAnalyzer
from src.utils.data_generator import DataGenerator
from src.models.session import InteractionRecord

# Imports de visualización
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import numpy as np
import pandas as pd

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

print("✅ Imports completados")

## 2. Generación de Datos Simulados

Generaremos datos de interacciones que simulan un paciente en UCI comunicándose mediante gestos.

In [ ]:
# Generar datos con estructura de comunidades
interactions = DataGenerator.generate_community_based(session_id="notebook_demo")

print(f"📊 Datos generados:")
print(f"  • Total de interacciones: {len(interactions)}")
print(f"  • Período de tiempo: {interactions[-1].timestamp - interactions[0].timestamp:.1f} segundos")
print(f"\n📝 Primeras 5 interacciones:")

for i, interaction in enumerate(interactions[:5], 1):
    print(f"  {i}. {interaction.from_node} → {interaction.to_node} (duración: {interaction.duration:.2f}s)")

## 3. Construcción del Grafo

In [ ]:
# Crear analizador y construir grafo
analyzer = InteractionGraphAnalyzer()
analyzer.build_from_interactions(interactions)

print(f"🔧 Grafo construido:")
print(f"  • Nodos (palabras): {analyzer.graph.number_of_nodes()}")
print(f"  • Aristas (transiciones): {analyzer.graph.number_of_edges()}")
print(f"  • Tipo: {'Dirigido' if analyzer.graph.is_directed() else 'No dirigido'}")
print(f"  • Débilmente conexo: {'Sí' if nx.is_weakly_connected(analyzer.graph) else 'No'}")

## 4. Análisis de Métricas Estructurales

In [ ]:
# Calcular todas las métricas
metrics = analyzer.compute_all_metrics()

print("📈 MÉTRICAS ESTRUCTURALES")
print("="*50)
print(f"Densidad: {metrics.density:.4f}")
print(f"Diámetro: {metrics.diameter}")
print(f"Camino promedio: {metrics.avg_path_length:.4f}" if metrics.avg_path_length else "Camino promedio: N/A")
print(f"Clustering promedio: {metrics.avg_clustering_coefficient:.4f}")

# Distribución de grados
degrees = [d for n, d in analyzer.graph.degree()]
print(f"\nDistribución de grados:")
print(f"  • Grado promedio: {np.mean(degrees):.2f}")
print(f"  • Grado máximo: {np.max(degrees)}")
print(f"  • Grado mínimo: {np.min(degrees)}")

In [ ]:
# Visualizar distribución de grados
plt.figure(figsize=(10, 6))
plt.hist(degrees, bins=10, edgecolor='black', alpha=0.7)
plt.xlabel('Grado del Nodo')
plt.ylabel('Frecuencia')
plt.title('Distribución de Grados en el Grafo de Interacciones')
plt.grid(True, alpha=0.3)
plt.show()

## 5. Análisis de Centralidades

Analizamos las diferentes medidas de centralidad para identificar nodos importantes.

In [ ]:
# Crear DataFrame con centralidades
centrality_data = []
for node_metric in metrics.node_metrics:
    centrality_data.append({
        'Nodo': node_metric.node_id,
        'Degree': node_metric.degree_centrality,
        'Betweenness': node_metric.betweenness_centrality,
        'Closeness': node_metric.closeness_centrality,
        'Eigenvector': node_metric.eigenvector_centrality,
        'PageRank': node_metric.pagerank
    })

df_centralities = pd.DataFrame(centrality_data)
df_centralities = df_centralities.sort_values('PageRank', ascending=False)

print("🎯 TOP 5 NODOS POR CENTRALIDAD")
print("="*80)
print(df_centralities.head())

In [ ]:
# Visualizar centralidades
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Análisis de Centralidades', fontsize=16, fontweight='bold')

centrality_cols = ['Degree', 'Betweenness', 'Closeness', 'Eigenvector', 'PageRank']

for idx, col in enumerate(centrality_cols):
    ax = axes[idx // 3, idx % 3]
    df_sorted = df_centralities.sort_values(col, ascending=True)
    
    ax.barh(df_sorted['Nodo'], df_sorted[col], color=sns.color_palette()[idx])
    ax.set_xlabel(col)
    ax.set_title(f'{col} Centrality')
    ax.grid(True, alpha=0.3)

# Ocultar el último subplot
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

## 6. Detección de Comunidades

Identificamos grupos de palabras que tienden a aparecer juntas.

In [ ]:
print(f"🌐 ANÁLISIS DE COMUNIDADES")
print("="*50)
print(f"Número de comunidades: {metrics.num_communities}")
print(f"Modularidad: {metrics.modularity_score:.4f}")
print(f"\nDetalles por comunidad:")

for community in metrics.communities:
    print(f"\nComunidad {community.community_id}:")
    print(f"  • Tamaño: {community.size} nodos")
    print(f"  • Nodos: {', '.join(community.nodes)}")
    print(f"  • Aristas internas: {community.internal_edges}")
    print(f"  • Aristas externas: {community.external_edges}")
    print(f"  • Contribución a modularidad: {community.modularity_contribution:.4f}")

In [ ]:
# Visualizar comunidades
plt.figure(figsize=(14, 10))

# Crear diccionario de colores por comunidad
community_colors = {}
for node_metric in metrics.node_metrics:
    community_colors[node_metric.node_id] = node_metric.community_id

# Layout
pos = nx.spring_layout(analyzer.graph, k=1.5, iterations=50, seed=42)

# Dibujar por comunidad
unique_communities = set(community_colors.values())
colors = sns.color_palette('husl', len(unique_communities))

for comm_id, color in zip(sorted(unique_communities), colors):
    nodes = [n for n, c in community_colors.items() if c == comm_id]
    nx.draw_networkx_nodes(
        analyzer.graph,
        pos,
        nodelist=nodes,
        node_color=[color],
        node_size=1500,
        label=f'Comunidad {comm_id}',
        alpha=0.9
    )

# Dibujar aristas
nx.draw_networkx_edges(
    analyzer.graph,
    pos,
    alpha=0.3,
    arrows=True,
    arrowsize=15
)

# Labels
nx.draw_networkx_labels(
    analyzer.graph,
    pos,
    font_size=10,
    font_weight='bold'
)

plt.title('Detección de Comunidades en el Grafo de Interacciones', fontsize=16, fontweight='bold')
plt.legend()
plt.axis('off')
plt.tight_layout()
plt.show()

## 7. Análisis de Robustez

Evaluamos qué tan robusto es el grafo ante la remoción de nodos críticos.

In [ ]:
print("🛡️ ANÁLISIS DE ROBUSTEZ")
print("="*50)
print(f"Nodos críticos: {', '.join(metrics.robustness.critical_nodes)}")
print(f"Vulnerability score: {metrics.robustness.vulnerability_score:.4f}")
print(f"\nEfecto de remover nodo más crítico:")
print(f"  • Camino promedio original: {metrics.robustness.avg_path_length_original:.4f}")
print(f"  • Camino promedio después: {metrics.robustness.avg_path_length_after_removal:.4f}")
print(f"  • Pérdida de conectividad: {metrics.robustness.connectivity_loss:.1%}")

## 8. Análisis de Transiciones

Analizamos los patrones de transición entre palabras.

In [ ]:
print("🔄 ANÁLISIS DE TRANSICIONES")
print("="*50)
print(f"Entropía de Shannon: {metrics.transitions.entropy:.4f}")
print(f"Burstiness: {metrics.transitions.burstiness:.4f}")
print(f"\nCaminos más comunes:")

for i, path in enumerate(metrics.transitions.most_common_paths[:5], 1):
    print(f"  {i}. {' → '.join(path)}")

In [ ]:
# Visualizar matriz de transiciones
transition_matrix = metrics.transitions.transition_matrix

# Convertir a matriz numpy
nodes = sorted(analyzer.graph.nodes())
n = len(nodes)
matrix = np.zeros((n, n))

for i, from_node in enumerate(nodes):
    if from_node in transition_matrix:
        for j, to_node in enumerate(nodes):
            if to_node in transition_matrix[from_node]:
                matrix[i, j] = transition_matrix[from_node][to_node]

# Visualizar
plt.figure(figsize=(12, 10))
sns.heatmap(
    matrix,
    xticklabels=nodes,
    yticklabels=nodes,
    annot=True,
    fmt='.2f',
    cmap='YlOrRd',
    cbar_kws={'label': 'Probabilidad de Transición'}
)
plt.title('Matriz de Transiciones', fontsize=16, fontweight='bold')
plt.xlabel('Nodo Destino')
plt.ylabel('Nodo Origen')
plt.tight_layout()
plt.show()

## 9. Modelos de Difusión

Simulamos cómo se difundiría información desde cada nodo.

In [ ]:
print("💫 ANÁLISIS DE DIFUSIÓN")
print("="*50)
print(f"Threshold de activación: {metrics.diffusion.activation_threshold:.2f}")
print(f"\nInfluence Maximizers (Top 3):")
for i, node in enumerate(metrics.diffusion.influence_maximizers, 1):
    spread = metrics.diffusion.spread_potential[node]
    cascade = metrics.diffusion.expected_cascade_size[node]
    print(f"  {i}. {node}: {spread:.1%} alcance, {cascade:.0f} nodos activados")

In [ ]:
# Visualizar potencial de difusión
spread_data = [(node, metrics.diffusion.spread_potential[node]) 
               for node in analyzer.graph.nodes()]
spread_data.sort(key=lambda x: x[1], reverse=True)

nodes, spreads = zip(*spread_data)

plt.figure(figsize=(12, 6))
plt.bar(nodes, spreads, color=sns.color_palette('viridis', len(nodes)))
plt.xlabel('Nodo')
plt.ylabel('Potencial de Difusión')
plt.title('Potencial de Difusión por Nodo (Independent Cascade Model)', fontsize=14, fontweight='bold')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.show()

## 10. Visualización del Grafo Completo

In [ ]:
# Visualización completa del grafo
plt.figure(figsize=(16, 12))

# Layout
pos = nx.spring_layout(analyzer.graph, k=2, iterations=50, seed=42)

# Tamaño de nodos según PageRank
pageranks = [m.pagerank for m in metrics.node_metrics]
node_sizes = [pr * 10000 for pr in pageranks]

# Color de nodos según comunidad
community_map = {m.node_id: m.community_id for m in metrics.node_metrics}
node_colors = [community_map[node] for node in analyzer.graph.nodes()]

# Dibujar nodos
nx.draw_networkx_nodes(
    analyzer.graph,
    pos,
    node_size=node_sizes,
    node_color=node_colors,
    cmap='tab10',
    alpha=0.9
)

# Grosor de aristas según peso
edges = analyzer.graph.edges()
weights = [analyzer.graph[u][v]['weight'] for u, v in edges]
max_weight = max(weights) if weights else 1

nx.draw_networkx_edges(
    analyzer.graph,
    pos,
    width=[w / max_weight * 5 for w in weights],
    alpha=0.4,
    edge_color='gray',
    arrows=True,
    arrowsize=20,
    connectionstyle='arc3,rad=0.1'
)

# Labels
nx.draw_networkx_labels(
    analyzer.graph,
    pos,
    font_size=12,
    font_weight='bold',
    font_color='white'
)

plt.title('Grafo de Interacciones DOCommunication\n(Tamaño = PageRank, Color = Comunidad, Grosor = Frecuencia)',
         fontsize=16, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

## Conclusiones

Este notebook ha demostrado un análisis completo del grafo de interacciones gestuales:

1. **Métricas Estructurales**: Evaluamos densidad, diámetro y clustering
2. **Centralidades**: Identificamos nodos clave usando 5 métricas diferentes
3. **Comunidades**: Detectamos grupos de palabras relacionadas
4. **Robustez**: Evaluamos vulnerabilidad del sistema
5. **Transiciones**: Analizamos patrones de flujo entre palabras
6. **Difusión**: Modelamos propagación de información

### Aplicaciones Prácticas

- **Optimización de vocabulario**: Priorizar palabras con alta centralidad
- **Diseño de UI**: Agrupar palabras por comunidades
- **Predicción**: Usar matriz de transiciones para sugerir próxima palabra
- **Robustez**: Identificar dependencias críticas del sistema